# Triangular lattice simulations

**authors:** Joseph Marcus
 
Here I simulate genetic data under the coalescent in a triangular lattice and explore the fit of different ways to compute expected genetic distances on simulated genotypes.

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
sys.path.append("../code/")
from habitat import *

# Setup the habitat

In [13]:
hab = TriangularLattice(8, 8)